In [2]:
from selenium import webdriver
import pandas as pd


We need to get the predicted fantasy points per game. This comes from the courtesy of http://rotoguru1.com/cgi-bin/fstats.cgi

We need to understand their API in order to create good enough queries so we can scrape them with selenium

The api seems to follow: week =1, game=fanduel, fd


we need to parse positions: 

In [10]:
driver = webdriver.Firefox()
# lets get week one, fanduel
driver.get('http://rotoguru1.com/cgi-bin/fyday.pl?week=1&game=fd') 


table = driver.find_element_by_xpath('/html/body/table/tbody/tr/td[3]/p[6]/table/tbody/tr/td[1]/table')
#('/html/body/center[1]/font/form/table/tbody/tr[1]/td[4]/input')

In [23]:
allRows =table.find_elements_by_xpath('/html/body/table/tbody/tr/td[3]/p[6]/table/tbody/tr/td[1]/table/tbody/tr')

So we need to make a pandas DF from this list. its probably better if we make different DataFrames for each role so we can see how they do over time in comparison, then join them if we need to, for the puposes of modelling. 

we're going to loop thru the scraped text of each of the rows, of each week, of each year. Sigh. gotta love text parsing. 
We'll make a list of positions: QB. RB, WR, TE, PK, Def. Pk = kickers.

Each time we hit the seperator, we increment the indexor and create a new Dataframe. 

Each dataframe will be named after a position. the columns will be: name ,team, opponent,points, FD salary




In [72]:
#make named DataFrames
QB = pd.DataFrame(columns=['name' ,'team', 'opponent','points', 'salary','week'])
RB = pd.DataFrame(columns=['name' ,'team', 'opponent','points', 'salary','week'])
WR = pd.DataFrame(columns=['name' ,'team', 'opponent','points', 'salary','week'])
TE = pd.DataFrame(columns=['name' ,'team', 'opponent','points', 'salary','week'])
PK = pd.DataFrame(columns=['name' ,'team', 'opponent','points', 'salary','week'])
Def = pd.DataFrame(columns=['name' ,'team', 'opponent','points', 'salary','week'])
# put Dataframes in list
DataFrames = [ QB,RB,WR,TE,PK,Def]


currentPosition = -1
skipNext = False
index = 0
for row in allRows:
    if skipNext == True:
        skipNext = False 
        continue
    if row.text == "Jump to:     QB   |   RB   |   WR   |   TE   |   PK   |   Def   |": 
        currentPosition += 1
        skipNext = True
        index = 0

    else:
        #add to list, legitimate player
        nameList = row.find_elements_by_xpath('td')
        name = nameList[0].text
        team = nameList[1].text
        opponent = nameList[2].text[2:]
        points = nameList[3].text
        salary = nameList[4].text[1:]
        week = 1
        if salary == "/A":
            salary = 0
        else:
            salary = nameList[4].text[1:]

            salary=int(salary.replace(',',''))
        
        
        appending = [name,team,opponent,points, salary, week]
        
        DataFrames[currentPosition].loc[index] = appending
        
        index += 1 

In [66]:
nameList = allRows[2].find_elements_by_xpath('td')
name = nameList[0]

team = nameList[1].text
opponent = nameList[2].text[2:]
points = nameList[3]
salary = nameList[4].text[1:]
salary=int(salary.replace(',',''))

print(team)

bal


In [59]:
for i in allRows[2].find_elements_by_xpath('td'):
    print(i.text)

Jackson, Lamar
bal
@ mia
33.56
$7,400


In [73]:
DataFrames[4]

,name,team,opponent,points,salary,week
0,"Butker, Harrison",kan,jac,17,0,1
1,"Gostkowski, Stephen",nwe,pit,16,0,1
2,"Lutz, Wil",nor,hou,15,0,1
3,"Zuerlein, Greg",lar,car,15,0,1
4,"Gonzalez, Zane",ari,det,14,0,1
5,"Santos, Cairo",ten,cle,13,0,1
6,"Gould, Robbie",sfo,tam,12,0,1
7,"Slye, Joey",car,lar,12,0,1
8,"Hopkins, Dustin",was,phi,11,0,1
9,"Prater, Matt",det,ari,11,0,1
